In [ ]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt

from multirtd.simple_planner import Simple_Planner
import multirtd.params as params

%load_ext autoreload
%autoreload 2

In [ ]:
# Initialize planner
p_0 = np.array([[-5], [0]])
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
planner = Simple_Planner(LPM_file, p_0)

In [ ]:
# Set planner goal and obstacles
planner.p_goal = np.array([[5], [0]])
# (center, radius) tuples
planner.obstacles = [(np.array([3, 1]), 1.5),
                     (np.array([0, -1.5]), 1.5),
                     (np.array([-3, 1]), 1.5)]

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

#ax.scatter(p_0[0], p_0[1], s=50, marker='^', color='b')
ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

plt.show()

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_0
v = np.zeros((2,1))
a = np.zeros((2,1))
# State history
P = []
planning_times = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    p = trajectory[0][:,traj_idx][:,None]
    v = trajectory[1][:,traj_idx][:,None]
    a = trajectory[2][:,traj_idx][:,None]
    P.append(p)
    traj_idx += 1

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P = np.hstack(P)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
# Visualize setup
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

ax.add_patch(plt.Circle(tuple(p_0), params.R_BOT, color='b'))
ax.scatter(planner.p_goal[0], planner.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

# Plot trajectory
ax.scatter(P[0,:], P[1,:], s=10, marker='o', color='b')

plt.show()

Dubin's planner

In [ ]:
from multirtd.dubins_planner import DubinsPlanner

In [ ]:
# Initialize planner
p_0 = np.array([-5, 0])
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
planner = DubinsPlanner(LPM_file, p_0)

# Set planner goal and obstacles
planner.p_goal = np.array([5, 0])
# (center, radius) tuples
planner.obstacles = [(np.array([3, 1]), 1.5),
                     (np.array([0, -1.5]), 1.5),
                     (np.array([-3, 1]), 1.5)]

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p = p_0
v = np.zeros(2)
a = np.zeros(2)
theta = 0
# State history
P = []; V = []; A = []
planning_times = []

speeds = []
omegas = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        start_t = time.time()
        trajectory = planner.replan((p,v,a))
        planning_times.append(time.time() - start_t)
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    p = trajectory[0][traj_idx]
    v = trajectory[1][traj_idx]
    a = trajectory[2][traj_idx]
    P.append(p)
    V.append(v)
    traj_idx += 1

    # speeds.append(trajectory[3])
    # thetas.append(trajectory[4])
    speeds.append(np.linalg.norm(v))
    omegas.append((np.arctan2(v[1], v[0]) - theta) / 0.1)
    theta = np.arctan2(v[1], v[0])

    if np.linalg.norm(p - planner.p_goal) < params.R_GOAL_REACHED:
        print("Goal reached")
        done = True

    t_sim += 1

P = np.array(P)
V = np.array(V)
print("Average planning time: ", np.mean(planning_times), " seconds")

In [ ]:
np.arctan2(V[2][1], V[2][0])

In [ ]:
omegas

In [ ]:
# plot speeds and thetas over time in side by side plots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
ax1.plot(speeds)
ax1.set_title('Speeds')
ax2.plot(omegas)
ax2.set_title('Omegas')
plt.show()


In [ ]:
dt = 0.1
x_0 = np.array([-5, 0, 0])
traj = []
for i in range(len(speeds)):
    traj.append(x_0)
    x_0 = x_0 + np.array([speeds[i]*np.cos(x_0[2]), speeds[i]*np.sin(x_0[2]), omegas[i]])*dt    

In [ ]:
traj = np.array(traj)
# Plot trajectory
plt.figure()
plt.plot(traj[:,0], traj[:,1])
plt.title("Trajectory")


## Multiple agents

In [ ]:
# Initialize planner
p_0_1 = np.array([[-5], [0]])
LPM_file = os.path.join(os.getcwd(),'..', 'models', 'quadrotor_linear_planning_model.mat')
planner_1 = Simple_Planner(LPM_file, p_0_1)
p_0_2 = np.array([[5], [-2]])
planner_2 = Simple_Planner(LPM_file, p_0_2)

# Set planner goal and obstacles
planner_1.p_goal = np.array([[5], [0]])
planner_2.p_goal = np.array([[-5], [-2]])
# (center, radius) tuples
planner_1.obstacles = [(np.array([3, 1]), 1.5),
                     (np.array([0, -1.5]), 1.5),
                     (np.array([-3, 1]), 1.5)]
planner_2.obstacles = [(np.array([3, 1]), 1.5),
                     (np.array([0, -1.5]), 1.5),
                     (np.array([-3, 1]), 1.5)]

In [ ]:
# "Controller" loop frequency 
# Determined by trajectory discretization
# Currently the controller just pops off setpoints from the planner trajectories to save for plotting
controller_hz = 10  # 0.1 s

# Planner loop frequency
# Determines time between replans
planner_hz = 2  # 0.5 s

t_sim = 0  # counter for simulation time
trajectory = None
traj_idx = 0
done = False

# Initialize state
p_1 = p_0_1
v_1 = np.zeros((2,1))
a_1 = np.zeros((2,1))
p_2 = p_0_2
v_2 = np.zeros((2,1))
a_2 = np.zeros((2,1))
# State history
P_1 = []
P_2 = []

while not done:

    # Planner
    if t_sim % int(controller_hz/planner_hz) == 0:
        # Replan
        print("Replanning: t_sim = ", t_sim)
        trajectory_1 = planner_1.replan((p_1,v_1,a_1))
        trajectory_2 = planner_2.replan((p_2,v_2,a_2))
        traj_idx = 0

    # Controller
    # Pop off next setpoint
    p_1 = trajectory_1[0][:,traj_idx][:,None]
    v_1 = trajectory_1[1][:,traj_idx][:,None]
    a_1 = trajectory_1[2][:,traj_idx][:,None]
    P_1.append(p_1)
    p_2 = trajectory_2[0][:,traj_idx][:,None]
    v_2 = trajectory_2[1][:,traj_idx][:,None]
    a_2 = trajectory_2[2][:,traj_idx][:,None]
    P_2.append(p_2)
    traj_idx += 1

    if (np.linalg.norm(p_1 - planner_1.p_goal) < params.R_GOAL_REACHED) and (np.linalg.norm(p_2 - planner_2.p_goal) < params.R_GOAL_REACHED):
        print("Goal reached")
        done = True

    t_sim += 1

P_1 = np.hstack(P_1)
P_2 = np.hstack(P_2)

In [ ]:
# Visualize 
fig, ax = plt.subplots(figsize=(12,6))
ax.set_xlim((-6, 6))
ax.set_ylim((-3, 3))
ax.grid()

# Start and goal
ax.add_patch(plt.Circle(tuple(p_0_1), params.R_BOT, color='b'))
ax.scatter(planner_1.p_goal[0], planner_1.p_goal[1], s=100, marker='*', color='g')
ax.add_patch(plt.Circle(tuple(p_0_2), params.R_BOT, color='b'))
ax.scatter(planner_2.p_goal[0], planner_2.p_goal[1], s=100, marker='*', color='g')

plt_obs = []
for obs in planner.obstacles:
    ax.add_patch(plt.Circle(tuple(obs[0]), obs[1], color='r'))

# Plot trajectory
ax.scatter(P_1[0,:], P_1[1,:], s=10, marker='o', color='b')
# Plot trajectory
ax.scatter(P_2[0,:], P_2[1,:], s=10, marker='o', color='b')

plt.show()